In [ ]:
import tkinter as tk
import requests
import json

class CountryInfoGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Country Info")
        self.root.geometry("400x300")
        
        self.countries = self.get_countries()
        self.selected_country = tk.StringVar()
        self.selected_country.set(self.countries[0])
        
        self.country_menu = tk.OptionMenu(self.root, self.selected_country, *self.countries, command=self.get_country_info)
        self.country_menu.pack(pady=10)
        
        self.country_info_label = tk.Label(self.root, text="", font=("Arial", 14))
        self.country_info_label.pack(pady=10)
        
        self.button_frame = tk.Frame(self.root)
        self.button_frame.pack()
        
        self.new_list_button = tk.Button(self.button_frame, text="New List", command=self.new_list)
        self.new_list_button.pack(side=tk.LEFT, padx=10)
        
    def get_countries(self):
        url = "https://restcountries.com/v2/all"
        response = requests.get(url)
        countries = json.loads(response.text)
        return [country["name"] for country in countries]
    
    def get_country_info(self, *args):
        country = self.selected_country.get()
        url = "https://restcountries.com/v2/name/{}?fullText=true".format(country)
        response = requests.get(url)
        country_data = json.loads(response.text)[0]
        
        info_text = "Country: {}\n\n".format(country_data["name"])
        info_text += "Capital: {}\n".format(country_data["capital"])
        info_text += "Population: {}\n".format(country_data["population"])
        info_text += "Region: {}\n".format(country_data["region"])
        info_text += "Subregion: {}\n".format(country_data["subregion"])
        info_text += "Languages: {}\n".format(", ".join(lang["name"] for lang in country_data["languages"]))
        self.country_info_label.configure(text=info_text)
        
    def new_list(self):
        self.countries = self.get_countries()
        self.selected_country.set(self.countries[0])
        self.country_menu['menu'].delete(0, 'end')
        for country in self.countries:
            self.country_menu['menu'].add_command(label=country, command=lambda name=country: self.selected_country.set(name))
        
if __name__ == "__main__":
    root = tk.Tk()
    app = CountryInfoGUI(root)
    root.mainloop()

In [ ]:
import tkinter as tk
from tkinter import ttk
import requests
import json

class WeatherForecastGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Weather Forecast")
        self.root.geometry("800x600")
        
        self.location_label = tk.Label(self.root, text="Enter a city name:")
        self.location_label.pack(pady=10)
        
        self.location_entry = tk.Entry(self.root, width=30, font=("Arial", 14))
        self.location_entry.pack(pady=10)
        self.location_entry.bind("<Return>", self.get_forecast)
        
        self.forecast_tree = ttk.Treeview(self.root, columns=("date", "description", "temp_min", "temp_max"), show="headings")
        self.forecast_tree.heading("date", text="Date")
        self.forecast_tree.heading("description", text="Description")
        self.forecast_tree.heading("temp_min", text="Min Temp")
        self.forecast_tree.heading("temp_max", text="Max Temp")
        self.forecast_tree.column("date", width=150)
        self.forecast_tree.column("description", width=300)
        self.forecast_tree.column("temp_min", width=100)
        self.forecast_tree.column("temp_max", width=100)
        self.forecast_tree.pack(pady=20)
        
    def get_forecast(self, event=None):
        city = self.location_entry.get()
        url = "https://api.openweathermap.org/data/2.5/forecast?q={}&units=metric&appid=YOUR_API_KEY".format(city) # replace YOUR_API_KEY with your actual API key
        response = requests.get(url)
        forecast = json.loads(response.text)
        
        self.forecast_tree.delete(*self.forecast_tree.get_children())
        for item in forecast["list"]:
            date = item["dt_txt"][:10]
            description = item["weather"][0]["description"]
            temp_min = "{:.0f}°C".format(item["main"]["temp_min"])
            temp_max = "{:.0f}°C".format(item["main"]["temp_max"])
            self.forecast_tree.insert("", "end", values=(date, description, temp_min, temp_max))
        
if __name__ == "__main__":
    root = tk.Tk()
    app = WeatherForecastGUI(root)
    root.mainloop()